In [1]:
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from openpyxl import Workbook
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import UnexpectedAlertPresentException

In [9]:
division = ['학부(신촌)', '대학원(신촌)', '의료원(신촌)', '학부(미래)', '학부(미래)', '대학원(미래)', '의료원(미래)']
college_dept = {
    "경영대학" : [
        "경영학전공"
    ]
}
# college_dept = {
#     # "교양기초(2019학번~)" : [
#     #     "채플", "기독교의이해", "글쓰기"
#     # ],
#     "대학교양(2019학번~)" : [
#         "문학과예술",
#         "인간과역사",
#         "언어와표현",
#         "가치와윤리",
#         "국가와사회",
#         "지역과세계",
#         "논리와수리",
#         "자연과우주",
#         "생명과환경",
#         "정보와기술",
#         "체육과건강"
#     ],
#     "자율선택(2019학번~)" : [
#         "자율선택"
#     ],
#     "RC교육(2019학번~)" : [
#         "사회참여(SE)",
#         "YONSEI RC1010",
#         "RC자기주도활동",
#         "RC심화"
#     ],
#     "공통기초(10~18학번)" : [
#         "기독교의이해",
#         "채플",
#         "글쓰기"
#     ],
#     "필수교양(10~18학번)" : [
#         "문학과예술",
#         "인간과역사",
#         "언어와표현",
#         "가치와윤리",
#         "국가와사회공동체",
#         "지역사회와세계",
#         "논리와수리",
#         "자연과우주",
#         "생명과환경",
#         "소프트웨어",
#     ],
#     "선택교양(10~18학번)" : [
#         "과학·기술영역",
#         "인문·예술영역",
#         "세계문화·언어영역",
#         "생활·건강영역",
#         "역사·철학영역",
#         "사회·윤리영역",
#         "Residential Education"    
#     ],
#     "문과대학" : [
#         "문과대학 공통",
#         "국어국문학전공",
#         "중어중문학전공",
#         "영어영문학전공",
#         "독어독문학전공",
#         "불어불문학전공",
#         "노어노문학전공",
#         "사학전공",
#         "철학전공",
#         "문헌정보학전공",
#         "심리학전공"
#     ],
#     "상경대학" : [
#         "경제학전공",
#         "응용통계학전공"
#     ],
#     "경영대학" : [
#         "경영학전공"
#     ],
#     "이과대학" : [
#         "이과대학 공통",
#         "수학전공",
#         "물리학전공",
#         "화학전공",
#         "지구시스템과학전공",
#         "천문우주학전공",
#         "대기과학전공"
#     ],
#     "공과대학" : [
#         "공과대학공통",
#         "기계공학전공",
#         "전기전자공학전공",
#         "신소재공학전공",
#         "건설환경공학전공",
#         "도시공학전공",
#         "건축공학전공",
#         "건축학(설계)",
#         "화공생명공학전공",
#         "산업공학전공",
#         "시스템반도체공학과",
#         "디스플레이융합공학과"
#     ],
#     "생명시스템대학" : [
#         "시스템생물학전공",
#         "생화학전공",
#         "생명공학전공",
#         "생명시스템공통"        
#     ],
#     "인공지능융합대학" : [
#         "컴퓨터과학과",
#         "인공지능학과",
#         "IT융합공학과",
#         "인공지능융합대학공통"
#     ],
#     "신과대학" : [
#         "신학과"
#     ],
#     "사회과학대학" : [
#         "사회학전공",
#         "사회과학대학공통",
#         "정치외교학전공",
#         "행정학전공",
#         "언론홍보영상학부",
#         "사회복지학전공",
#         "문화인류학전공"
#     ],
#     "음악대학" : [
#         "음악대학공통",
#         "교회음악전공",
#         "성악전공",
#         "피아노전공",
#         "관현악전공",
#         "작곡전공"
#     ],
#     "교육과학대학" : [
#         "교육학전공",
#         "체육교육학전공",
#         "스포츠응용산업전공"
#     ],
#     "생활과학대학" : [
#         "의류환경학전공",
#         "식품영양학전공",
#         "실내건축학전공",
#         "아동·가족전공",
#         "통합디자인전공"
#     ],
#     "언더우드국제대학" : [
#         "공통교과과정(신촌)",
#         "언더우드학부(인문사회)-비교문학과문화",
#         "언더우드학부(인문사회)-경제학",
#         "언더우드학부(인문사회)-국제학",
#         "언더우드학부(공학)-생명과학공학",
#         "언더우드학부(인문사회)-정치외교학",
#         "융합사회과학부-사회정의리더십",
#         "융합사회과학부-계량위험관리",
#         "융합사회과학부-과학기술정책",
#         "융합사회과학부-지속개발협력",
#         "융합과학공학부-나노과학공학",
#         "융합과학공학부-에너지환경융합",
#         "융합과학공학부-바이오융합",
#         "테크노아트학부-정보·인터랙션디자인",
#         "아시아학부-아시아학",
#         "테크노아트학부-창의기술경영",
#         "테크노아트학부-문화디자인경영"
#     ],
#     "약학대학" : [
#         "약학전공"
#     ],
#     "글로벌인재대학" : [
#         "글로벌인재학부-한국문화전공",
#         "글로벌인재학부-한국언어문화교육전공",
#         "글로벌인재학부-문화미디어전공",
#         "글로벌인재학부-바이오생활공학전공",
#         "글로벌인재학부-응용정보공학전공",
#         "GLD공통교과과정"    
#     ],
#     "의·치·간(1학년)" : [
#         "의예과(1학년)",
#         "치의예과(1학년)",
#         "간호(1학년)"
#     ],
#     "연계전공" : [
#         "유럽지역학",
#         "미국학",
#         "한국및동아시아학(중국학)",
#         "한국및동아시아학(한국학)",
#         "한국및동아시아학(일본학)",
#         "외교통상학",
#         "벤처학",
#         "리더십",
#         "디지털예술학",
#         "한국및동아시아학(동아시아학)",
#         "비교문학",
#         "문화비평학",
#         "공공리더십"
#     ],
#     "ROTC" : [
#         "군사학"
#     ],
#     "교직과정" : [
#         "교직과정"
#     ],
#     "평생교육사과정" : [
#         "평생교육사과정"
#     ],
#     "Study Abroad Course" : [
#         "Undergraduate",
#         "Graduate",
#         "Korean Language"
#     ],
#     "공통" : [
#         "창업,현장실습"
#     ],
#     "학부기초(~2009학번)" : [
#         "기독교의이해"
#     ],
#     "학부필수(~2009학번)" : [
#         "인간의이해",
#         "자연의이해",
#         "사회의이해",
#         "문화의이해",
#         "세계의이해"
#     ],
#     "계열기초(~2009학번)" : [
#         "계열기초"
#     ],
#     "학부선택(~2009학번)" : [
#         "인간의이해와역사",
#         "과학과기술",
#         "사회와가치",
#         "문학과예술",
#         "외국어와세계이해",
#         "건강과스포츠"
#     ],
#     "(2019~2023)국제캠퍼스" : [
#         "교양기초-채플"
#     ],
#     "(2011~2018)국제캠퍼스" : [
#         "공통기초-채플"
#     ]
# }

semesters = ['2023-2학기', '2023-1학기'] # Semester options

In [3]:
def modify_course_info_dict(course_info_json : dict) -> dict:
    desired_data = dict()
    
    key = list(course_info_json.keys())[0]
    tmp_data = course_info_json[key][0]
        
    for key, value in tmp_data.items():
            if str(key) == "campsDivNm": # 캠퍼스 구분
                desired_data['캠퍼스'] = str(value)
            elif str(key) == "subjtnbNo":
                desired_data['학정번호'] = str(value)
            elif str(key) == "subjtNm": # 과목명
                desired_data['과목명'] = str(value)
            elif str(key) == "cdt": # 학점
                desired_data['학점'] = str(value)
            elif str(key) == "cgprfNm": # 담당교수명
                desired_data['담당교수명'] = str(value)
            elif str(key) == "lctreTimeNm": # 강의 시간
                desired_data['강의시간'] = str(value)
            elif str(key) == "lecrmNm": # 강의실 
                desired_data['강의실'] = str(value)
            elif str(key) == "atnlcPercpCnt": # 수강정원
                desired_data['수강정원'] = str(value)
            elif str(key) == "cnt":
                desired_data['참여인원'] = str(value)
            elif str(key) == "mjrprPercpCnt": # 전공자 정원
                desired_data['전공자정원'] = str(value)
            elif str(key) == "sy1PercpCnt": # 1학년 정원
                desired_data['1학년-정원'] = str(value)
            elif str(key) == "sy2PercpCnt":
                desired_data['2학년-정원'] = str(value)
            elif str(key) == "sy3PercpCnt": # 3학년 정원
                desired_data['3학년-정원'] = str(value)
            elif str(key) == "sy4PercpCnt": # 4학년 정원
                desired_data['4학년-정원'] = str(value)
            elif str(key) == "sy5PercpCnt": # 5학년 정원
                desired_data['5학년-정원'] = str(value)
            elif str(key) == "sy6PercpCnt": # 6학년 정원
                desired_data['6학년-정원'] = str(value) 
            elif str(key) == "usePosblMaxMlgVal": # 최대 사용 가능 마일리지
                desired_data['최대사용가능마일리지'] = str(value)
            elif str(key) == "minMlg": # 마일리지 최소값
                desired_data['마일리지-최솟값'] = str(value)
            elif str(key) == "maxMlg": # 마일리지 최댓값
                desired_data['마일리지-최댓값'] = str(value)
            elif str(key) == "avgMlg": # 마일리지 평균값
                desired_data['마일리지-평균값'] = str(value)
            elif str(key) == "gradeEvlMthdDivCd": # 성적 평가 방법 구분 코드 | 상대:0 | 절대:!
                desired_data['성적평가방식'] = '절대평가' if (value == '1') else '상대평가'
            elif str(key) == "estblDeprtCd": # 학과 코드
                desired_data['학과코드'] = str(value)
            elif str(key) == "syy": # year
                desired_data['연도'] = str(value)
            elif str(key) == "cgprfEngNm": # 교수명(영문)
                desired_data['영문교수명'] = str(value)
            elif str(key) == "subjtEngNm":
                desired_data['영문과목명'] = str(value)
            
    return desired_data        
    
def modify_mileage_info(rank_info_json : dict) -> list:
    mileage_info_list = list()

    key = list(rank_info_json.keys())[0]
    tmp_data_list = rank_info_json[str(key)]
        
    for tmp_data in tmp_data_list:
        tmp_dict = dict()
        tmp_dict['순위'] = tmp_data['mlgRank']
        tmp_dict['마일리지'] = tmp_data['mlgVal']
        tmp_dict['특수교육대상자/(장애학생)여부'] = tmp_data['dsstdYn']
        tmp_dict['전공자/(복수전공자)'] = tmp_data['mjsbjYn']
        tmp_dict['신청과목수'] = tmp_data['aplySubjcCnt']
        tmp_dict['졸업신청'] = tmp_data['grdtnAplyYn']
        tmp_dict['초수강여부'] = tmp_data['fratlcYn']
        tmp_dict['총이수학점비율'] = tmp_data['ttCmpsjCdtRto']
        tmp_dict['직전학기이수학점비율'] = tmp_data['jstbfSmtCmpsjCdtRto']
        tmp_dict['학년'] = tmp_data['hy']
        mileage_info_list.append(tmp_dict)

    return mileage_info_list

In [4]:
def get_into_class(driver):
    class_btn = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[5]/div/div[1]/div/div/div/div/div[1]/div/div/div[1]/div/div/div[3]/div/div/div[1]/div/div/div/a')   
    class_btn.click()
    time.sleep(3)
    course_page = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[5]/div/div[1]/div/div/div/div/div[1]/div/div/div[2]/div/div/div[1]/div/div/div[2]/div/div/div/div[3]/div/div/div[1]/div/div[3]/div')
    course_page.click()
    driver.implicitly_wait(3)

In [5]:
def enter_portal(driver):
    # User info
    USER_ID = 'user_id'
    USER_PW = 'user_pw'

    # Open chrome
    driver.get(url)
    driver.implicitly_wait(5)

    # Login field
    id_field = driver.find_element(By.ID, 'loginId')
    pw_field = driver.find_element(By.ID, 'loginPasswd')
    login_btn = driver.find_element(By.ID, 'loginBtn')

    id_field.clear()
    id_field.send_keys(USER_ID)
    pw_field.clear()
    pw_field.send_keys(USER_PW)
    login_btn.click()
    time.sleep(4)
    get_into_class(driver=driver)
    



In [6]:
url = 'https://underwood1.yonsei.ac.kr/com/lgin/SsoCtr/initPageWork.do?'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach', True)
chrome_options.add_argument('--enable-logging')
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
wait = WebDriverWait(driver=driver, timeout=10)
action = webdriver.ActionChains(driver)
enter_portal(driver=driver)

In [ ]:
final_data_collection_sinchon = {
        "학부(신촌)" : []
    }

print(list(college_dept.keys()))
for key in list(college_dept.keys()):
    for dept in college_dept[str(key)]:
        driver.get(url)
        time.sleep(4)
        get_into_class(driver=driver)
        time.sleep(5)
        
        college_input = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[1]/div/div/div[2]/div/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div[1]/div/div/div[3]/div/div[1]/div/div/div/div/div[3]/div/div/div/div[6]/div/div[1]/input')
        dept_input = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[1]/div/div/div[2]/div/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div[1]/div/div/div[3]/div/div[1]/div/div/div/div/div[3]/div/div/div/div[8]/div/div[1]/input')
        search_btn = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[1]/div/div/div[2]/div/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div[1]/div/div/div[3]/div/div[1]/div/div/div/div/div[3]/div/div/div/div[14]')

        try :
            # Enter search options
            college_input.click()
            college_input.send_keys(Keys.CONTROL + 'A')
            college_input.send_keys(str(key)) # Send option value to college input field
            college_input.send_keys(Keys.ENTER)
            time.sleep(1)
            dept_input.click()
            time.sleep(2)

            dept_input.send_keys(Keys.CONTROL + 'A')
            dept_input.send_keys(str(dept)) # Send option value to department input field
            dept_input.send_keys(Keys.ENTER)
            time.sleep(1)
            search_btn.click()
            time.sleep(3)

            mileage_elems = driver.find_elements(By.XPATH, '/html/body/div[1]/div[1]/div/div[1]/div/div/div[2]/div/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div[1]/div/div/div[3]/div/div[1]/div/div/div/div/div[4]/div/div/div/div[3]/div/div/div[3]/div/div[2]/div/div[1]/div/div/div/div[6]')
            for mileage in mileage_elems:
                mileage.click()
                time.sleep(2)
                # Enter semester options
                for semester in semesters:
                    semester_input = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[1]/div/div/div[2]/div/div[1]/input')
                    semester_input.click()
                    semester_input.send_keys(Keys.CONTROL + 'A')
                    semester_input.send_keys(str(semester))
                    semester_input.send_keys(Keys.ENTER)
                    time.sleep(2)

                    driver.execute_cdp_cmd('Network.enable', {})
                
                    # Capture network log entries
                    log_entries = driver.get_log("performance")
                    
                    result_dict = dict()
                    result_dict['연도-학기'] = semester
                    for entry in log_entries:
                        try:
                            message_obj = json.loads(entry.get("message"))
                            message = message_obj.get("message")
                            method = message.get("method")
                            
                            modified_course_info_json = dict()
                            modified_rank_info = list()

                            if method == 'Network.responseReceived':
                                response_url = message.get('params',{}).get('response',{}).get('url','')
                                request_id = message.get('params', {}).get('requestId', '')
                                
                                # Get course information
                                if 'findMlgAppcsResltList.do' in response_url:
                                    # Get course information response
                                    response = driver.execute_cdp_cmd('Network.getResponseBody',{'requestId':request_id})
                                    course_info_response_body = response.get('body', '')
                                    course_info_json = json.loads(course_info_response_body) # convert response to json(dict in python)
                                    key = list(course_info_json.keys())[0]
                                    if len(course_info_json[str(key)]) == 0:
                                        print("no data")
                                        continue
                                    else:
                                        modified_course_info_json = modify_course_info_dict(course_info_json)
                                        result_dict = modified_course_info_json
                                # Get mileage information  
                                if "findMlgRankResltList.do" in response_url:
                                    response = driver.execute_cdp_cmd('Network.getResponseBody',{'requestId':request_id})
                                    rank_info_response_body = response.get('body','')
                                    rank_info_json = json.loads(rank_info_response_body)
                                    modified_rank_info = modify_mileage_info(rank_info_json)
                                    result_dict['마일리지-순위'] = modified_rank_info
                                    result_dict['학부'] = str(key)
                                    result_dict['학과'] = str(dept)
                        except Exception as e:
                            print(e)
                    if len(list(result_dict.keys())) > 1:
                        print(result_dict)
                        final_data_collection_sinchon['학부(신촌)'].append(result_dict)
                        print(final_data_collection_sinchon)
                # Close mileage tab
                close_btn = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div[1]/div[1]/div[2]')
                close_btn.click()
                driver.implicitly_wait(2)
        except UnexpectedAlertPresentException:
            alert = driver.switch_to.alert
            alert.accept()
with open('result.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(final_data_collection_sinchon, ensure_ascii=False, indent=4))